# RAPPOR

## Settings

These are the settings for the RAPPOR algorithm itself:

In [1]:
k = 128
h = 2
m = 100
f = 0
p = 0.65
q = 0.35

The reported values can either be hashed using `md5` or `sha256`.
In Google's repository, `md5` is used. For generated datasets, the choice shouldn't really matter. For custom datasets, it's important to choose the same hash function that was also used for the data collection.

In [2]:
hash_function = ["md5", "sha256"][0]

### Option 1: Data Generation

You can either automatically let this notebook generate data, or load an existing dataset.

In [3]:
n = 1000000
M = 100
distribution = ["normal", "exponential", "uniform", "zipf1", "zipf1.5"][0]

### Option 2: Loading an existing dataset

If you already have a dataset that you want to load, change this flag to `False`:

In [4]:
generate_data = True

`clients` should then be a Python list that contains tuples.
The first element for each tuple is a numpy array that contains the reported bits. The second element is an integer that describes which cohort the respective user is assigned to.

In [5]:
clients = []

### Candidates

If you want to check for specific values, `candidates` should be a list of them.

In [6]:
candidates = []

If your dataset also contains the true counts, `true_counts` can be a list with the true counts for the given candidate strings.

In [7]:
true_counts_available = False
true_counts = []

If the dataset is automatically generated, `true_counts` is filled with the correct data and `candidates` defaults to all reported values.

---

## Hash function

In [8]:
sc.addPyFile("client/rappor.py")
sc.addPyFile("client/hmac_drbg.py")

In [9]:
from rappor import get_bloom_bits as get_bloom_bits_md5
from hashlib import sha256

In [10]:
def get_bloom_bits_sha256(value, cohort, h, k):
    bits = []
    
    for hi in range(h):
        seed = str(cohort) + str(hi)
        digest = sha256(seed + value).digest()

        bit = ord(digest[-1]) % k
        bits.append(bit)

    return bits

In [11]:
hash_functions = {
    "sha256": get_bloom_bits_sha256,
    "md5": get_bloom_bits_md5
}

In [12]:
if hash_function in hash_functions:
    get_bloom_bits = hash_functions[hash_function]
else:
    raise NotImplementedError("Unimplemented hash function")

## Data Generation

### Distributions

In [13]:
import numpy as np
from functools import partial
from scipy.stats import rv_discrete

In [14]:
def sample_normal(n, M):
    return np.floor(np.random.normal(M / 2, M / 6, size=(n)))

In [15]:
def sample_uniform(n, M):
    return np.floor(np.random.uniform(0, M, size=(n)))

In [16]:
def sample_exponential(n, M):
    return np.floor(np.random.exponential(scale=M/5, size=(n)))

In [17]:
def sample_custom_zipf(s, n, M):
    pdf = 1. / np.array(range(1, M))**float(s)
    pdf = pdf / pdf.sum()
    distribution = rv_discrete(name='zipf1', values=(range(len(pdf)), pdf))
    return distribution.rvs(size=n)

def sample_zipf(s):
    return partial(sample_custom_zipf, s)

While it doesn't happen often, the distributions above can generate values that are not between $0$ and $M$. In this case, we filter them out and resample new values until we have $n$ valid values.

In [18]:
def filter_out_of_bounds(seq, lower, upper):
    seq = seq[seq >= lower]
    seq = seq[seq < upper]
    return seq

In [19]:
def sample(n, M, distribution=sample_normal):
    data = distribution(n, M)
    data = filter_out_of_bounds(data, 0, M)
    
    while len(data) < n:
        additional_data = distribution(n - len(data), M)
        additional_data = filter_out_of_bounds(additional_data, 0, M)
        data = np.append(data, additional_data)
    
    return data

### Candidate Generation

In [20]:
def generate_candidates(M):
    return ["v%d" % i for i in range(1, M + 1)]

In [21]:
if len(candidates) == 0:
    candidates = generate_candidates(M)

In [22]:
distribution_map = {
    "normal": sample_normal,
    "exponential": sample_exponential,
    "uniform": sample_uniform,
    "zipf1": sample_zipf(1),
    "zipf1.5": sample_zipf(1.5)
}

In [23]:
used_distribution = distribution_map[distribution]
indices = sample(n, M, distribution=used_distribution)

In [24]:
reported_values = [candidates[int(i)] for i in indices]

### Assignment to cohorts

We can reuse the sampling functions we create earlier! Here, all users are assigned to cohorts uniformly randomly. The same logic is used in the shield study.

In [25]:
cohorts = map(int, sample(n, m, distribution=sample_uniform))

### Generating user reports

In [26]:
def build_bloom_filter((reported_value, cohort)):
    set_bits = get_bloom_bits(reported_value, cohort, h, k)
    
    bits = np.zeros(k)
    bits[set_bits] = 1
    
    return bits, cohort

The individual bits are flipped according to Bernoulli distributions with probabilities $f, p, q$.
Because numpy doesn't have helpers for these, we use the equivalent binomial distributions with $n = 1$.

In [27]:
def bernoulli(p, size):
    return np.random.binomial(n=1, p=p, size=(size))

In [28]:
def build_prr((bits, cohort)):
    randomized_bits = np.where(bernoulli(f, k))[0]
    bits[randomized_bits] = bernoulli(0.5, len(randomized_bits))
    return bits, cohort

In [ ]:
def build_irr((bits, cohort)):
    result = np.zeros(k)
    set_bits = np.where(bits == 1)[0]
    unset_bits = np.where(bits == 0)[0]
    
    result[set_bits] = bernoulli(q, len(set_bits))
    result[unset_bits] = bernoulli(p, len(unset_bits))
    
    return result, cohort

In [ ]:
if generate_data:
    rdd = sc.parallelize(zip(reported_values, cohorts))
    rdd = rdd.map(build_bloom_filter).map(build_prr).map(build_irr)
    clients = rdd.collect()

### True counts

In [ ]:
if generate_data:
    true_counts = np.zeros(M)
    idx, counts = np.unique(indices, return_counts=True)
    idx = map(int, idx)
    true_counts[idx] = counts
    
    true_counts_available = True

## Analysis

### Summing

Individual user reports are not very useful to us, instead we need to sum up how often each bit position was reported.

We're using the variable conventions from the paper here. $N$ is a vector containing the number of reports from the individual cohorts. $c$ is a matrix
where $c_{ij}$ tells us how often bit $j$ was set in cohort $i$.

In [ ]:
c = np.zeros((m, k))
N = np.zeros(m)

for bits, cohort in clients:
    c[cohort] += bits
    N[cohort] += 1
    
c = c.T

### Target values `y `

In [ ]:
def estimate_bloom_count(c, N):
    Y = c - ((p + 0.5 * f * q - 0.5 * f * p) * N)
    Y /= ((1 - f) * (q - p))
    return Y

In [ ]:
def get_target_values(c, N):
    Y = estimate_bloom_count(c, N)
    return (Y / N).T.reshape(k * m)

In [ ]:
y = get_target_values(c, N)

### Data matrix `X`

In [ ]:
def get_features(candidates):
    matrix = []

    for cohort in range(m):
        rows = []

        for candidate in candidates:
            bits = np.zeros(k)
            bits_set = get_bloom_bits(candidate, cohort, h, k)
            bits[bits_set] = 1
            rows.append(bits)

        for row in np.array(rows).T:
            matrix.append(row)

    X = np.array(matrix)
    
    return X

In [ ]:
X = get_features(candidates)

### Fitting

In [ ]:
from scipy.optimize import nnls

In [ ]:
def fit(X, y):
    x0, _ = nnls(X, y)
    return x0

In [ ]:
params = fit(X, y)

### Significance test

In [ ]:
from scipy.stats import t
from numpy.linalg import inv, norm

In [ ]:
significance_level = 0.05
bonferroni_corrected_level = significance_level / M

In [ ]:
predictions = X.dot(params)
num_datapoints, num_features = X.shape
MSE = norm(y - predictions, ord=2)**2 / (num_datapoints - num_features)

In [ ]:
var = MSE * inv(X.T.dot(X)).diagonal()
sd = np.sqrt(var)
ts = params / sd

In [ ]:
degrees_of_freedom = num_datapoints - 1
p_values = np.array([2 * (1 - t.cdf(np.abs(i), degrees_of_freedom)) for i in ts])

In [ ]:
significant_i = np.where(p_values <= bonferroni_corrected_level)[0]
significant = params[significant_i]

In [ ]:
analyzed = np.zeros(M)
analyzed[significant_i] = significant
estimates = analyzed * N.sum()

## Presenting the results

### Listed

In [ ]:
from pandas import DataFrame

In [ ]:
def create_estimate_df(candidates, estimates, original, true_counts_available):
    indices = np.argsort(estimates)[::-1]
    reported_candidates = [candidates[i] for i in indices]
    reported_estimates = np.array(estimates[indices], dtype=np.int32)
    
    columns = ["Candidate", "Estimated count"]
    
    if true_counts_available:
        reported_original = np.array(original[indices], dtype=np.int32)
        data = np.array(zip(reported_candidates, reported_estimates, reported_original))
        columns.append("Actual count")
    else:
        data = np.array(zip(reported_candidates, reported_estimates))

    df = DataFrame(data=data)
    df.columns = columns
    return df

In [ ]:
create_estimate_df(candidates, estimates, true_counts, true_counts_available).head(15)

### Visually

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
handles = []
labels = []

if true_counts_available:
    original_bar = plt.bar(range(M), true_counts, width=1., color='orange', edgecolor='darkorange', alpha=0.6)
    handles.append(original_bar)
    labels.append("True")
    
reported_bar = plt.bar(range(M), estimates, width=1., color='blue', edgecolor='darkblue', alpha=0.6)
handles.append(reported_bar)
labels.append("Estimated")

plt.title("RAPPOR results")
plt.legend(handles, labels, prop={'size': 8})
plt.xlabel("Index of candidate string")
plt.ylabel("Count")
plt.show()